In [ ]:
%pip install opencv-python
%pip install scikit-learn
%pip install tensorflow
%pip install matplotlib
import tensorflow as tf
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Parameters
CLASSES = ["apple", "calculator", "water_bottle", "food_box"]  # 4 classes from paper
IMG_SIZE = (8, 8)  # Target ultra-low resolution
DATA_PATH = "../../../dataset/rgbd-dataset"  # Base path to your dataset


def get_bbox_from_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    x,y,w,h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    return x,y,w,h


def load_depth_maps(class_names, data_path):
    images, labels = [], []
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(data_path, class_name)
        
        for instance_folder in sorted(os.listdir(class_path)):
            instance_path = os.path.join(class_path, instance_folder)
            if not os.path.isdir(instance_path):
                continue

            # Process each _depth.png and match with its _mask.png
            for file in sorted(os.listdir(instance_path)):
                if file.endswith("_depthcrop.png"):
                    base_prefix = file.replace("_depthcrop.png", "")
                    depth_path = os.path.join(instance_path, file)
                    mask_path = os.path.join(instance_path, f"{base_prefix}_maskcrop.png")

                    # Check if both files exist
                    if not os.path.isfile(depth_path) or not os.path.isfile(mask_path):
                        print(f"Missing pair for {file}")
                        continue

                    # Get bounding box from corresponding mask
                    bbox = get_bbox_from_mask(mask_path)
                    print(file)
                    print(mask_path)
                    if not bbox:
                        print(f"Invalid or empty mask for {mask_path}")
                        continue
                    x, y, w, h = bbox

                    # Load and crop depth image
                    depth_img = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)
                    if depth_img is None or depth_img.shape == ():
                        print(f"Invalid depth image at {depth_path}")
                        continue

                    height, width = depth_img.shape
                    x1, y1 = max(0, x), max(0, y)
                    x2, y2 = min(width, x + w), min(height, y + h)

                    if x2 <= x1 or y2 <= y1:
                        print(f"Invalid bbox in {depth_path}")
                        continue

                    cropped = depth_img[y1:y2, x1:x2]
                    resized = cv2.resize(cropped, IMG_SIZE, interpolation=cv2.INTER_AREA)
                    normalized = resized.astype('float32') / 4000.0  # Adjust if needed

                    images.append(normalized)
                    labels.append(label)
    
    return np.array(images), np.array(labels)

    
    

# Load the data
print("Loading data...")
X, y = load_depth_maps(CLASSES, DATA_PATH)
X = X.reshape(-1, 8, 8, 1)  # Reshape for CNN input




In [ ]:
# Añadir padding mirror para primera etapa de Conv
#Formato de la matriz de entrada X[k][i][j]
# donde k es la imagen, i es la fila de la imagen y j es la columna

N = X.shape[0]
padded_list = []

for k in range(N):

    mat = X[k].squeeze().copy()  # shape: (8, 8)

    padded_rows = []
    for i in range(8):
        row = mat[i]
        new_row = np.insert(row, 0, row[0])             # duplica primer valor
        new_row = np.insert(new_row, len(new_row), row[-1])  # duplica último valor
        padded_rows.append(new_row)
    
    mat_8x10 = np.array(padded_rows)

    # Padding de filas (arriba y abajo)
    top_row = mat_8x10[0]
    bottom_row = mat_8x10[-1]
    mat_10x10 = np.vstack([top_row, mat_8x10, bottom_row])

    padded_list.append(mat_10x10)

# Convertir lista a array numpy de shape (N, 10, 10)
X_padded = np.array(padded_list)
X_padded=X_padded.reshape(-1, 10, 10, 1)

In [ ]:

# Split data (70% train, 15% val, 15% test)
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

print("\nData loaded successfully!")
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")

In [ ]:
print("Label counts:", np.unique(y, return_counts=True))

In [ ]:
import tensorflow as tf


model = tf.keras.Sequential([
    # Input: 8x8x1 (normalized depth maps)
    tf.keras.layers.Conv2D(
        filters=3,                  # 3 kernels (paper: "3 channels")
        kernel_size=(3, 3),         # 3x3 convolutions
        activation='relu',
        padding='valid',             # Preserves spatial dimensions (8x8)
        input_shape=(10, 10, 1)       # Input shape
    ),
    tf.keras.layers.Dropout(0.25),  # 25% dropout (paper: after each conv layer)
    
    # Second conv layer
    tf.keras.layers.Conv2D(
        filters=4,                  # 4 kernels (paper: "4 channels")
        kernel_size=(3, 3),
        activation='relu',
        padding='valid'             # No padding → output shrinks to 6x6
    ),
    tf.keras.layers.Dropout(0.25),
    
    # Classifier
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        units=4,                    # 4 classes (apple, calculator, water_bottle, food_box)
        activation='softmax'
    )
])
#'''

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
y_train_classes = np.argmax(y_train, axis=1)
#model
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),  # SGD (not Adam!) for stability
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
# Compute class weights
best_acc=0
best_weights = [0,0]

class_weights = {0:4.0,
                1:5.1,
                2:2.0,
                3:2.0}

# Convert to dictionary format expected by Keras
#class_weights = dict(enumerate(class_weights))

history = model.fit(
    X_train, y_train,
    epochs=300,                     # Paper used 300 epochs
    batch_size=32,                  # Paper: 32 for 4 classes
    validation_data=(X_val, y_val),
    class_weight=class_weights,     # Your computed weights
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch, lr: lr * 0.5 if (epoch + 1) % 15 == 0 else lr  # LR decay every 15 epochs
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=6,
            restore_best_weights=True
        )
    ]
)
last_train_acc = history.history['accuracy'][-1]
if best_acc<=last_train_acc:
    best_acc = last_train_acc

print("Mejor precision:",best_acc)
#print("Mejor peso water bottle:",best_weights[0])
#print("Mejor peso food box:", best_weights[1])

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()


In [ ]:
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix
%pip install seaborn
import seaborn as sns

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_classes = np.argmax(y_test, axis=1)

# Check class distribution in true labels
print("True class counts:", np.bincount(y_true_classes))

# Check class distribution in predictions
print("Predicted class counts:", np.bincount(y_pred_classes))

# Check raw confusion matrix (counts)
print("Raw CM (counts):\n", confusion_matrix(y_true_classes, y_pred_classes))





class_names=["apple","calculator","food box","water bottle"]
# Compute confusion matrix (raw counts)
cm = confusion_matrix(y_true_classes, y_pred_classes)

# Convert to percentages (row-wise: % of predictions per true class)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Plot
plt.figure(figsize=(8, 6))
plt.imshow(cm_percent, interpolation='nearest', cmap='Blues')
plt.colorbar(format='%.2f%%')

# Add labels
class_names = ['apple', 'calculator', 'water_bottle', 'food_box']  # Replace with your class names
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Annotate cells with percentages
thresh = cm_percent.max() / 2.
for i in range(cm_percent.shape[0]):
    for j in range(cm_percent.shape[1]):
        plt.text(j, i, f'{cm_percent[i, j]:.1f}%',
                 horizontalalignment="center",
                 color="white" if cm_percent[i, j] > thresh else "black")

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix (%)')
plt.tight_layout()
plt.show()

In [ ]:
unique, counts = np.unique(y_true_classes, return_counts=True)
print("Class distribution:", dict(zip(class_names, counts)))

In [ ]:

first_conv_layer = model.layers[0]  # Index 0 is your first Conv2D layer

# Get the weights (kernels and biases) of the first layer
first_layer_weights = first_conv_layer.get_weights()

# The weights are returned as a list where:
# first_conv_layer_weights[0] contains the kernels (filters)
# first_conv_layer_weights[1] contains the biases
kernels_first_layer = first_layer_weights[0]
biases_first_layer = first_layer_weights[1]

print("First layer kernel:", kernels_first_layer)
# Should print (3, 3, 1, 3) because:
# (kernel_height, kernel_width, input_channels, output_channels)

# Similarly for the second convolutional layer
second_conv_layer = model.layers[2]  # Index 2 (after Dropout)
second_layer_weights = second_conv_layer.get_weights()
kernels_second_layer = second_layer_weights[0]

print("Second layer kernels shape:", kernels_second_layer.shape)
# Should print (3, 3, 3, 4) because:
# (kernel_height, kernel_width, input_channels (from previous layer), output_channels)

In [ ]:
from tensorflow.keras.models import Model
import numpy as np

# 1. Configuración
i = 1  # Índice del ejemplo
layer_index = 0  # Primera capa

# 2. Crear submodelo
first_layer_model = Model(
    inputs=model.inputs,  # Usar model.input (singular)
    outputs=model.layers[layer_index].output
)

# 3. Obtener datos
single_input = X_test[i:i+1].astype('float32')  # Shape (1, 10, 10, 1)
activations = first_layer_model.predict(single_input)  # Shape (1, 8, 8, 3)

biases = model.layers[0].get_weights()[1]   # shape: (3,)

# 4. Mostrar entrada
print("=== Entrada (10x10) ===")
print(f"Ejemplo {i} - Shape: {single_input.shape}")
print((single_input[0,:,:,0]))  # Mostrar matriz 10x10 con 3 decimales

# 5. Mostrar activaciones para cada filtro
print("\n=== Activaciones Capa Conv2D (8x8) ===")
print(f"Capa: {model.layers[layer_index].name} - Filtros: {activations.shape[-1]}")
for j in range(activations.shape[-1]):
    print(f"\nFiltro {j+1}:")
    print(activations[0,:,:,j])  # Matriz 8x8 con 4 decimales
    print(f"Valor máximo: {activations[0,:,:,j].max():.4f}")
    print(f"Valor mínimo: {activations[0,:,:,j].min():.4f}")
    print(f"\nValor del bias: ",biases[j])
for i, layer in enumerate(model.layers):
    weights = layer.get_weights()
    if weights:
        print(f"\nLayer {i} - {layer.name}")
        print("Weights shape:", [w.shape for w in weights])
        
        kernel = weights[0]
        
        if len(kernel.shape) == 4:  # Conv2D layer
            # kernel shape: (height, width, input_channels, output_channels)
            h, w, in_channels, out_channels = kernel.shape
            print(f"Kernel shape: {kernel.shape} (H x W x In_Ch x Out_Ch)")
            
            # Show first 3x3 kernel for first input channel and first output filter
            example_kernel = kernel[:, :, 0, 0]  # select input=0, output=0 filter
            print("Example 3x3 kernel matrix (input channel 0 -> output channel 0):")
            print((example_kernel, 4))
            example_kernel = kernel[:, :, 0, 1]  # select input=0, output=0 filter
            print("Example 3x3 kernel matrix (input channel 0 -> output channel 1):")
            print((example_kernel, 4))
            example_kernel = kernel[:, :, 0, 2]  # select input=0, output=0 filter
            print("Example 3x3 kernel matrix (input channel 0 -> output channel 1):")
            print((example_kernel, 4))


In [ ]:
import numpy as np
from tensorflow.keras.models import Model

# Get the output of the first conv layer
model_layer_0 = Model(inputs=model.inputs, outputs=model.layers[0].output)

# Pick an input
i = 1
input_image = X_test[i:i+1]  # shape: (1, 10, 10, 1)
input_2d = input_image[0, :, :, 0]  # shape: (10, 10)

# Get kernel and bias from filter 0
kernels = model.layers[0].get_weights()[0]  # shape: (3, 3, 1, 3)
biases = model.layers[0].get_weights()[1]   # shape: (3,)

kernel0 = kernels[:, :, 0, 0]  # filter 0
bias0 = biases[0]
print("Biases;\n")
print(biases,"\n")
# Manual convolution
manual_output = np.zeros((8, 8))
for row in range(8):
    for col in range(8):
        patch = input_2d[row:row+3, col:col+3]
        manual_output[row, col] = np.sum(patch * kernel0) + bias0

manual_output_relu = np.maximum(manual_output, 0)  # Apply ReLU

# Get model prediction
activations = model_layer_0.predict(input_image)[0]
actual_output = activations[:, :, 0]

# Compare
print("Manual (with bias + ReLU):")
print(manual_output_relu)

print("\nModel output:")
print(actual_output)

print("\nDifference:")
print(np.abs(manual_output_relu - actual_output))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Asegúrate de tener la lista CLASSES definida
CLASSES = ["apple", "calculator", "water_bottle", "food_box"]

for i in range(len(X_test)):
    pred_class = np.argmax(preds[i])
    true_class = np.argmax(y_test[i])
    
    if pred_class == true_class:
        plt.imshow(X_test[i].squeeze(), cmap='gray')
        plt.title(f"True: {CLASSES[true_class]}, Pred: {CLASSES[pred_class]}")
        plt.axis('off')
        plt.show()